# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math
from IPython.display import display
from IPython.core.debugger import Tracer

###### HELPERS #############################################
######
def drawImgFromArray(image, isgray):
    plt.figure()
    if isgray:
        plt.imshow(image, cmap='gray')
    else:
        plt.imshow(image)

def drawHist(hist, bins):
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    plt.figure()
    plt.bar(center, hist, align='center', width=width)
    plt.show()
    
def smallline(line, length=6):
    return abs(line[0] - line[2]) < length and abs(line[1] - line[3]) < length

def reject_outliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

def within_acceptance(anum, amean, astd, m=2):
    return abs(anum - amean) < m * astd

def within_bin(anum,amin,amax):
    return anum >= amin and anum <= amax

def pointsInLines(lines):
    xs=[]
    ys=[]
    for line in lines:
        xs.append(line[0][0])
        xs.append(line[0][2])
        ys.append(line[0][1])
        ys.append(line[0][3])
        
    return {'x':xs, 'y':ys}

###### FILTERS #############################################
######
def filter_by_bin_and_deviation (lines, slopes, intercepts, minm, maxm, minb, maxb):
    std_slopes = np.std(slopes)
    std_intercepts = np.std(intercepts)
    mean_slope = np.mean(slopes)
    mean_intercept = np.mean(intercepts)

    cmpdata = {
        'prior': {
            'std_slopes': std_slopes,
            'std_intercepts': std_intercepts,
            'linecount': len(lines),
            'mean_slopes': mean_slope,
            'mean_intercepts': mean_intercept
        }
    }

    newlines = []
    newslopes = []
    newyinter = []
    for i, ln in enumerate(lines):
#       if within_acceptance(slopes[i], mean_slope, std_slopes, 1)
        if within_bin(slopes[i], minm, maxm) and within_acceptance(intercepts[i], mean_intercept, std_intercepts, 1):
            newlines.append(lines[i])
            newslopes.append(slopes[i])
            newyinter.append(intercepts[i])

    cmpdata['post'] = {
        'std_slopes': std_slopes,
        'std_intercepts': std_intercepts,
        'linecount': len(newlines),
        'mean_slopes': np.mean(newslopes),
        'mean_intercepts': np.mean(newyinter)
    }

    return newlines, newslopes, newyinter, cmpdata


def filter_by_binning (lines, slopes, intercepts, minm, maxm, minb, maxb):
    std_slopes = np.std(slopes)
    std_intercepts = np.std(intercepts)

    cmpdata = {
        'filter': {
            'bin_slopes_min': minm,
            'bin_slopes_max': maxm,
            'bin_intercepts_min': minb,
            'bin_intercepts_max': maxb
        },
        'prior': {
            'std_slopes': std_slopes,
            'std_intercepts': std_intercepts,
            'linecount': len(lines),
            'mean_slopes': np.mean(slopes),
            'mean_intercepts': np.mean(intercepts)
        }
    }

    newlines = []
    newslopes = []
    newyinter = []
    for i, ln in enumerate(lines):
        if within_bin(slopes[i], minm, maxm) and within_bin(intercepts[i], minb, maxb):
            newlines.append(lines[i])
            newslopes.append(slopes[i])
            newyinter.append(intercepts[i])

    cmpdata['post'] = {
        'std_slopes': np.std(newslopes),
        'std_intercepts': np.std(newyinter),
        'linecount': len(newlines),
        'mean_slopes': np.mean(newslopes),
        'mean_intercepts': np.mean(newyinter)
    }

    return newlines, newslopes, newyinter, cmpdata


###### Throw out junky lines #############################################
######
def filterlines(lines):
    
    l_lines=[]
    r_lines=[]
    l_slopes=[]
    r_slopes=[]
    l_intercepts=[]
    r_intercepts=[]
    l_intercept_avg=0
    r_intercept_avg=0
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            if smallline(line[0]):
                continue

            slope = (y2-y1)/(x2-x1)

            if np.isinf(slope) or np.isnan(slope):
                continue

            if slope < 0:
                l_lines.append(line)
                l_slopes.append(slope)
                l_intercepts.append(-x1 * slope + y1)
            else:
                r_lines.append(line)
                r_slopes.append(slope)
                r_intercepts.append(-x1 * slope + y1)

    
    ## Do some auto binning in attempt to find the "strongest" lines
    l_hist_slopes, l_bins_slopes = np.histogram(l_slopes, bins='auto')
    r_hist_slopes, r_bins_slopes = np.histogram(r_slopes, bins='auto')

    l_hist_intercepts, l_bins_intercepts = np.histogram(l_intercepts, bins='auto')
    r_hist_intercepts, r_bins_intercepts = np.histogram(r_intercepts, bins='auto')
    
    l_max_slope_count = l_hist_slopes.argmax(axis=0)
    r_max_slope_count = r_hist_slopes.argmax(axis=0)
    
    l_max_intercept_count = l_hist_intercepts.argmax(axis=0)
    r_max_intercept_count = r_hist_intercepts.argmax(axis=0)
    
    # append a value to represent "unlimited" as last hist bin
    np.append(l_bins_slopes, 999999999)
    np.append(l_bins_intercepts, 999999999)

    # Do Filter
    l_lines_f, l_slopes_f, l_intercepts_f, l_cmpdata = filter_by_bin_and_deviation(
        l_lines, l_slopes, l_intercepts,
        l_bins_slopes[l_max_slope_count], l_bins_slopes[l_max_slope_count + 1],
        l_bins_intercepts[l_max_intercept_count], l_bins_intercepts[l_max_intercept_count+1])

    r_lines_f, r_slopes_f, r_intercepts_f, r_cmpdata = filter_by_bin_and_deviation(
        r_lines, r_slopes, r_intercepts,
        r_bins_slopes[r_max_slope_count], r_bins_slopes[r_max_slope_count + 1],
        r_bins_intercepts[r_max_intercept_count], r_bins_intercepts[r_max_intercept_count+1])
    
    
    return l_lines_f, r_lines_f, l_cmpdata, r_cmpdata
        
def grayscale(img):
    """Applies the Grayscale transform"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
    """
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    llines, rlines, lcmpdata, rcmpdata = filterlines(lines)
    
    lslope = lcmpdata['post']['mean_slopes']
    rslope = rcmpdata['post']['mean_slopes']
    lyinter = lcmpdata['post']['mean_intercepts']
    ryinter = rcmpdata['post']['mean_intercepts']
        
    
    def x_for_y(y, yint, slope):
        try:
            xfactor = 1 / slope
            return int((y - yint) * xfactor)
        except Exception:
            return -100
    
    imgh = len(img)
    imgc = int(imgh / 2 + 70)
    
    
    def getnewval(old, new, weight=1, thres=0.07):
        if (np.isnan(new)):
            return old
        if abs((old+0.00001 - new) / old) < thres:
            return (old*weight + new) / (weight+1)
        else:
            return old
        
    def getnewline(old, new, weight=2,thres=0.10):
        points=[]
        for i in range(0,4):
            points.append(int(getnewval(old[i], new[i], weight, thres)))
        return points
        
        
    lline = [x_for_y(imgc, lyinter, lslope), imgc, x_for_y(imgh, lyinter, lslope), imgh]
    rline = [x_for_y(imgc, ryinter, rslope), imgc, x_for_y(imgh, ryinter, rslope), imgh]
    
    if draw_lines.lslope == -99999:
        draw_lines.lslope = lslope
        draw_lines.rslope = rslope
        draw_lines.lyinter = lyinter
        draw_lines.ryinter = ryinter
        draw_lines.avglline = np.copy(lline)
        draw_lines.avgrline = np.copy(rline)
    else:
#         draw_lines.lslope = getnewval(draw_lines.lslope, lslope)
#         draw_lines.rslope = getnewval(draw_lines.rslope, rslope)
#         draw_lines.lyinter = getnewval(draw_lines.lyinter, lyinter)
#         draw_lines.ryinter = getnewval(draw_lines.ryinter, ryinter)
        draw_lines.avglline = getnewline(draw_lines.avglline, lline)
        draw_lines.avgrline = getnewline(draw_lines.avgrline, rline)
    

    
    cv2.line(img, (draw_lines.avglline[0], draw_lines.avglline[1]), (draw_lines.avglline[2], draw_lines.avglline[3]), color, thickness)
    cv2.line(img, (draw_lines.avgrline[0], draw_lines.avgrline[1]), (draw_lines.avgrline[2], draw_lines.avgrline[3]), color, thickness)

draw_lines.lslope=-99999

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

run your solution on all test_images and make copies into the test_images directory).

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.
    
def pipeline(startimg):
    # Define our parameters for Canny
    low_threshold = 50
    high_threshold = 150

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = (np.pi*1)/180 # angular resolution in radians of the Hough grid
    threshold = 20    # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 40 #minimum number of pixels making up a line
    max_line_gap = 20   # maximum gap in pixels between connectable line segments

    #define offset from mask center (for including turns in non-masked area)
    apex_offset = 50
    
    image = np.copy(startimg)
#     drawImgFromArray(original, 0)
    
    # convert, canny and blur
    image = grayscale(image)
    image = canny(image, low_threshold, high_threshold)
    image = gaussian_blur(image, 5)    
#     drawImgFromArray(image, 1)
    
    # mask 
    imshape = image.shape
    vertices = np.array([[(0+50,imshape[0]),(imshape[1]/2-apex_offset, 340), (imshape[1]/2+apex_offset, 300), (imshape[1]-50,imshape[0])]], dtype=np.int32)
    image = region_of_interest(image, vertices)
#     drawImgFromArray(image, 1)

    #hough
    line_image = np.copy(image)*0 # creating a blank to draw lines on
    hufflines = hough_lines(image, rho, theta, threshold, min_line_len, max_line_gap)
#     drawImgFromArray(image, 1)
    
    line_img = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, hufflines)
    
    # weighted image
    image = weighted_img(line_img, startimg)
#     drawImgFromArray(image, 1)

    return image


# for fn in os.listdir("test_images/"):
#     #read image
#     original = (mpimg.imread('test_images/'+fn)*255).astype('uint8')
#     image = pipeline(original)
#     drawImgFromArray(image, 1)
#     mpimg.imsave('test_images/copy_'+fn, image)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    result = pipeline(image)
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
draw_lines.lslope=-99999
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!



Potential Improvements...
1. process color channels independently to avoid any loss of useful data during grayscale conversion
2. use curve fitting for line drawing
3. track line endpoints independently to apply modified filtering for forground/background points (since foreground experience much quicker changes)
4. detect all lane lines and only use the ones closest to the centerline.


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
draw_lines.lslope=-99999
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))